<a href="https://colab.research.google.com/github/MFox72/Rainfall/blob/main/Rainfall_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title RUN -- Imports
# Import the data from google drive

import cv2 as cv
import datetime
from google.colab import drive
from google.colab.patches import cv2_imshow
import numpy as np
drive.mount("/content/gdrive")
# No need for these now, data is stored in TransData in cropped form
#!unzip /content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/Data.zip
#!unzip /content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/Data2.zip
#!unzip /content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/Data3.zip
#!unzip /content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransCroppedSoFar
!unzip /content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransCroppedAll.zip

# For testing whether the image conversion will work with 1 year as it didnt with 2
#!unzip /content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/2021.zip


Streaming output truncated to the last 5000 lines.
  inflating: TransCropped/2023-06-30-08.jpg  
  inflating: TransCropped/2023-06-29-22.jpg  
  inflating: TransCropped/2023-06-24-05.jpg  
  inflating: TransCropped/2023-06-30-03.jpg  
  inflating: TransCropped/2023-06-24-06.jpg  
  inflating: TransCropped/2023-06-24-09.jpg  
  inflating: TransCropped/2023-06-24-07.jpg  
  inflating: TransCropped/2023-06-29-17.jpg  
  inflating: TransCropped/2023-06-29-16.jpg  
  inflating: TransCropped/2023-06-24-04.jpg  
  inflating: TransCropped/2023-06-29-15.jpg  
  inflating: TransCropped/2023-06-29-19.jpg  
  inflating: TransCropped/2023-06-29-14.jpg  
  inflating: TransCropped/2023-06-29-18.jpg  
  inflating: TransCropped/2023-06-24-01.jpg  
  inflating: TransCropped/2023-06-23-22.jpg  
  inflating: TransCropped/2023-06-24-02.jpg  
  inflating: TransCropped/2023-06-30-01.jpg  
  inflating: TransCropped/2023-06-30-02.jpg  
  inflating: TransCropped/2023-06-24-03.jpg  
  inflating: TransCropped/202

In [ ]:
#@title DONT RUN -- Read, import and rename new data
######## ONLY USED FOR SHAPING NEW DATA - DO NOT RUN ########
# Setup loop start, to iterate through all days, to crop and rename
# Then save in a different folder

root = '/content/Data 3/ScreenShot-'
startDay = datetime.datetime(2022, 1, 1)
endDay = datetime.datetime(2024, 1, 1)
delta = datetime.timedelta(days=1)
gmt = True
flag0 = False #to indicate 00h needs to be the same, but 2h needs -
flag1 = False #to indicate 00h needs to be the same, but 1h needs +
count = 0     #to check whether all 8774 imgs are accounted for - 2020 = 366*24 (-10 due to missing data on 14th Dec)

while startDay != endDay:

    year = str(startDay.year)
    month = str(startDay.month)
    day = str(startDay.day)

    if len(month) < 2:
        month = '0' + month
    if len(day) < 2:
        day = '0' + day

    for hour in range(24):

        # check for clocks going forward
        if month == '03' and startDay.strftime('%A') == 'Sunday' and int(day) >= 25 and hour == 0:
            flag0 = True
            gmt = False
            print('daylight taken!')

        # check for clocks going backward
        if month == '10' and startDay.strftime('%A') == 'Sunday' and int(day) >= 25 and hour == 0:
            flag1 = True
            gmt = True
            print('Daylight saved!')

        #############################################-- BST --################################################
        # If working in BST, need to add 1 to hour when retrieving file but name it as it would be in GMT (UTC)
        if gmt == False:
            # Account for the lost hour
            adjustH = (hour + 1) % 24

            if len(str(hour)) < 2:
                hour = '0' + str(hour)
            if len(str(adjustH)) < 2:
                adjustH = '0' + str(adjustH)

            # If it is the day the clocks change, the first hour is as for GMT, and there is a special case
            if flag0 == True:
                if str(hour) == '00':
                    img = cv.imread(f"{root}{year}-{month}-{day} {hour}_00_00.jpg")
                    cropped = img[143:470, 518:788]
                    cv.imwrite(f'/content/gdrive/My Drive/Bath/Dissertation/RainfallProject/TransData20223/{year}-{month}-{day}-{hour}.jpg', cropped)
                    count += 1
                    flag0 = False

            else:
                # If adjustH is 00, increase day and month where applicable
                if adjustH == '00':
                    adjustDay = startDay + delta

                    adjustD = str(adjustDay.day)
                    adjustM = str(adjustDay.month)

                    if len(adjustD) < 2:
                        adjustD = '0' + adjustD
                    if len(adjustM) < 2:
                        adjustM = '0' + adjustM

                    img = cv.imread(f"{root}{year}-{adjustM}-{adjustD} 00_00_00.jpg")
                    try:
                        cropped = img[143:470, 518:788]
                        cv.imwrite(f'/content/gdrive/My Drive/Bath/Dissertation/RainfallProject/TransData20223/{year}-{month}-{day}-{hour}.jpg', cropped)
                        count += 1
                    except:
                        print(f"Nonetype error1: {year}-{month}-{day}-{hour}")

                else:
                    # Read the image one hour above GMT
                    # Account for 2 hours of missing data in 2019
                    if year == '2022' and month == '10' and day == '09' and int(hour) <= 22:
                        continue

                    else:
                        img = cv.imread(f"{root}{year}-{month}-{day} {adjustH}_00_00.jpg")
                        try:
                            cropped = img[143:470, 518:788]
                            cv.imwrite(f'/content/gdrive/My Drive/Bath/Dissertation/RainfallProject/TransData20223/{year}-{month}-{day}-{hour}.jpg', cropped)
                            count += 1
                        except:
                            print(f"Nonetype error2: {year}-{month}-{day}-{hour}")


        #############################################-- GMT --################################################
        if gmt == True:
            if len(str(hour)) < 2:
                hour = '0' + str(hour)

            if flag1 == True:
                img = cv.imread(f"{root}{year}-{month}-{day} 01_00_00 (1).jpg")
                cropped = img[143:470, 518:788]
                cv.imwrite(f'/content/gdrive/My Drive/Bath/Dissertation/RainfallProject/TransData20223/{year}-{month}-{day}-{hour}.jpg', cropped)
                count += 1
                flag1 = False

            else:
                # Account for 10 hours of missing data in 2020
                if year == '2020' and month == '12' and day == '14' and int(hour) <= 9:
                    continue

                else:
                    img = cv.imread(f"{root}{year}-{month}-{day} {hour}_00_00.jpg")
                    try:
                        cropped = img[143:470, 518:788]
                        cv.imwrite(f'/content/gdrive/My Drive/Bath/Dissertation/RainfallProject/TransData20223/{year}-{month}-{day}-{hour}.jpg', cropped)
                        count += 1
                    except:
                        print(f"Nonetype error3: {year}-{month}-{day}-{hour}")


    startDay += delta

if count == 17445:
    print(f'Yippee! {count}')
else:
    print(f':( {count}')


In [ ]:
#@title DONT RUN -- Crop images further
### Dont run
import os
root1 = '/content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransData'

cCount = 0

for sc in os.listdir(root1):
    cCount += 1
    img = cv.imread(f'{root1}/{sc}')
    cropped = img[170:310,105:245]
    cv.imwrite(f'/content/gdrive/My Drive/Bath/Dissertation/RainfallProject/TransCropped/{sc}', cropped)

print(cCount)


In [ ]:
#@title RUN* -- Load image paths to splits - Data overlaps - Run this or below
# 17534 in 2020-2021, 26203 in 2019, 2022-23 - 43737
import os
import random

#root2 = '/content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransCroppedSoFar'
#root3 = '/content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransData'
#root4 = '/content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransData2019'
#root5 = '/content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransData20223'
#root6 = '/content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransCropped'
root7 = '/content/TransCropped'
tCount = 0
preImgs = 5
pathList = []

for scrnSht in os.listdir(root7):
    tCount += 1
    pathList.append(scrnSht)

### Sort all imgs, randomise distribution into splits
pathList.sort()
imgSet = pathList[preImgs:]
random.Random(0).shuffle(imgSet)
testLabels = imgSet[-6500:]
valLabels = imgSet[-13000:-6500]
trainLabels = imgSet[:-13000]

### Create sets of x input imgs and y target imgs
xTrain = []
xVal = []
xTest = []

for imgIndex in range(len(trainLabels)):

    trainInputs = []
    trainIndex = pathList.index(trainLabels[imgIndex])

    if imgIndex < len(testLabels):

        ### Add set of input images for val and test targets at index
        valInputs = []
        valIndex = pathList.index(valLabels[imgIndex])
        testInputs = []
        testIndex = pathList.index(testLabels[imgIndex])

        for inputIndex in range(preImgs, 0, -1):
            valInputs.append(pathList[valIndex-inputIndex])
            testInputs.append(pathList[testIndex-inputIndex])
            trainInputs.append(pathList[trainIndex-inputIndex])

        xVal.append(valInputs)
        xTest.append(testInputs)
        xTrain.append(trainInputs)

    else:

        for inputIndex in range(preImgs, 0, -1):
            trainInputs.append(pathList[trainIndex-inputIndex])

        xTrain.append(trainInputs)


trainSet = [xTrain, trainLabels]
valSet = [xVal, valLabels]
testSet = [xTest, testLabels]

#print(tCount, len(xTrain), len(xVal), len(xTest), len(imgSet))

In [ ]:
#@title RUN* -- Load image paths to splits - No data overlapping - Run this or above
# 17534 in 2020-2021, 26203 in 2019, 2022-23 - 43737
import os
import random

root7 = '/content/TransCropped'
tCount = 0
preImgs = 5
pathList = []

for scrnSht in os.listdir(root7):
    tCount += 1
    pathList.append(scrnSht)

### Chunk all images (if divides) into chunks of preImgs images + target image
pathList.sort()
chunkedList = []

for img in range(int(len(pathList)/(preImgs+1))):
    chunk = []
    for t in range(preImgs+1):
        chunk.append(pathList[img*(preImgs+1)+t])
    chunkedList.append(chunk)

### Randomise and split
random.Random(0).shuffle(chunkedList)
partition = int(len(chunkedList) * 0.15)
testPaths = chunkedList[-partition:]
valPaths = chunkedList[-(2 * partition):-partition]
trainPaths = chunkedList[:-(2 * partition)]

pathList = pathList[:-(len(pathList)%(preImgs+1))]

#print(len(testPaths), len(valPaths), len(trainPaths), len(valPaths[0]))

### Create sets of x input imgs and y target imgs
xTrain = []
yTrain = []
xVal = []
yVal = []
xTest = []
yTest = []

for chunkIndex in range(len(trainPaths)):

    trainInputs = []

    if chunkIndex < len(testPaths):

        ### Add set of input images targets at index
        valInputs = []
        testInputs = []

        for inputIndex in range(preImgs):
            valInputs.append(valPaths[chunkIndex][inputIndex])
            testInputs.append(testPaths[chunkIndex][inputIndex])
            trainInputs.append(trainPaths[chunkIndex][inputIndex])

        xVal.append(valInputs)
        yVal.append(valPaths[chunkIndex][preImgs])
        xTest.append(testInputs)
        yTest.append(testPaths[chunkIndex][preImgs])
        xTrain.append(trainInputs)
        yTrain.append(trainPaths[chunkIndex][preImgs])

    ### Same but only for train when others are done
    else:

        for inputIndex in range(preImgs):
            trainInputs.append(trainPaths[chunkIndex][inputIndex])

        xTrain.append(trainInputs)
        yTrain.append(trainPaths[chunkIndex][preImgs])


trainSet = [xTrain, yTrain]
valSet = [xVal, yVal]
testSet = [xTest, yTest]

#print(trainSet)

#print(tCount, len(testPaths), len(valPaths), len(trainPaths))
#print(len(yVal), len(xVal))

In [ ]:
#@title RUN -- Define Dataset Class
### Create dataset of images

import torch
from torch.utils.data import Dataset
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms

class CroppedImagesDataset(Dataset):
    def __init__(self, data, directory):
        self.data = data[0]
        self.labels = data[1]
        self.img_dir = directory
        self.transform = transforms.Compose([transforms.Resize((48,48))])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        ### Read target image
        img_path = os.path.join(self.img_dir, self.labels[idx])
        image = torch.Tensor.float(read_image(img_path))
        #image = read_image(img_path)

        ### Read input images
        inputImgs = []
        for input in range(len(self.data[idx])):
            inpImgPath = os.path.join(self.img_dir, self.data[idx][input])
            inputImgs.append(torch.Tensor.float(read_image(inpImgPath)))
            #inputImgs.append((read_image(inpImgPath)))

        if self.transform:
            image = self.transform(image)
            for img in range(len(inputImgs)):
                inputImgs[img] = self.transform(inputImgs[img])

        return image, inputImgs


In [ ]:
#@title RUN -- Define Split Datasets
### Define datasets and respective dataloaders
from torch.utils.data import DataLoader

trainDataset = CroppedImagesDataset(trainSet, root7)
trainDataloader = DataLoader(trainDataset, batch_size=4, shuffle=True)

valDataset = CroppedImagesDataset(valSet, root7)
valDataloader = DataLoader(valDataset, batch_size=4, shuffle=True)

testDataset = CroppedImagesDataset(testSet, root7)
testDataloader = DataLoader(testDataset, batch_size=4, shuffle=True)


In [ ]:
#@title TEST dataset cell
import matplotlib.pyplot as plt
targetImg, dataImgs = next(iter(trainDataloader))
print(targetImg.type())
#print(targetImg)
#print(dataImgs)
print(f"Feature batch shape: {targetImg.size()}")
print(f"Data batch shape: {dataImgs[0].size()}, {len(dataImgs)}")
img = targetImg[0].squeeze()
#print(img.size())
#label = dataImgs[0]
#p = transforms.Compose([transforms.Resize((48,48))])
#smallImg = p(img) #torch.nn.functional.interpolate(targetImg[0], ([48, 48]))
plt.imshow(img.permute(1, 2, 0))
#plt.imshow(smallImg.permute(1, 2, 0))
plt.show()
#print(f"Label: {label}")

#for i, data in enumerate(trainDataloader):
#    print(i)
    #print(dataImgs)


In [ ]:
#@title 1 - ConvLSTM classes
# from github https://github.com/ndrplz/ConvLSTM_pytorch/blob/master/convlstm.py
import torch.nn as nn
import torch


class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):

    """

    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.

    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [ ]:
#@title 1 - Create instance of ConvLSTM
from numpy import asarray
import matplotlib.pyplot as plt

model = ConvLSTM(input_dim=3,
                 hidden_dim=[64, 32, 3],
                 kernel_size=(3,3),
                 num_layers=3,
                 batch_first=True,
                 bias=True,
                 return_all_layers=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

dataRoot = '/content/gdrive/MyDrive/Bath/Dissertation/RainfallProject/TransCroppedSoFar/'


In [ ]:
#@title 1 - Simple training ConvLSTM
# Train the ConvLSTM
import torch.optim as optim
import torchvision

np.random.seed(1)

def trainModel():

    model.train()

    loss = nn.MSELoss()
    optimiser = optim.Adam(model.parameters(), lr = 0.0001)

    epochs = 300
    batchSize = 4
    height = 140
    width = 140
    print("successful start")

    for epoch in range(epochs):

        ### Randomise a batch of 16 images from the training set and create tensors for them
        randImgs = np.random.randint(0, len(trainPath), batchSize)
        batch = np.zeros([batchSize, 5, 3, height, width], dtype="float32")
        targets = np.zeros([batchSize, 3, height, width], dtype="float32")

        ### Read in the randomised images and the 4 next hour images for each of the 16
        ### Also read in the target - the 6th hour in the chain - change this for longer predictions
        for start in range(len(randImgs)):

            for hour in range(5):
                image = cv.imread(dataRoot + pathList[pathList.index(trainPath[randImgs[start]])+hour])
                #cv2_imshow(image)
                image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
                image = np.moveaxis(image, -1, 0)
                batch[start][hour] = asarray(image)
                if (epoch % 30 == 0 or epoch+1 == epochs) and hour == 0:
                    print(pathList[pathList.index(trainPath[randImgs[start]])+hour])

            targetImg = cv.imread(dataRoot + pathList[pathList.index(trainPath[randImgs[start]])+5])
            #cv2_imshow(targetImg)
            targetImg = cv.cvtColor(targetImg, cv.COLOR_BGR2RGB)
            targetImg = np.moveaxis(targetImg, -1, 0)
            targets[start] = asarray(targetImg)

        batch = torch.from_numpy(batch)
        targets = torch.from_numpy(targets)


        ### Define loss and optimiser

        optimiser.zero_grad()
        outputs, last = model.forward(batch)

        #for target in range(len(targets)):
        #    #print(last[0][0].size())
        #    passLoss = loss(last[0][target], targets[target])
#
        #    passLoss.backward()
#
        #    optimiser.step()
        #    #print(last[0][target][0])
        #    print(passLoss)
        #    img = last[0][0][0].detach().numpy().transpose(1, 2, 0)
        #    plt.imshow(img)
        #    plt.show()

        passLoss = loss(last[0][0], targets[0])

        passLoss.backward()

        optimiser.step()

        if epoch % 5 == 0 or epoch+1 == epochs:

            print(passLoss)

        #img = last[0][0][0].detach().numpy().transpose(1, 2, 0)
        #plt.imshow(img)
        #plt.show()

            torchvision.utils.save_image(last[0][0][0], f'/content/SCs2/{epoch}.jpg')



In [ ]:
#@title 1 - Call train function
trainModel()


In [ ]:
#@title 2 - VAE imports and functions

### Maybe pixels should be limited to only the colours used in the input images?
### MNIST VAEs use an input and output dimensionality of 784 (28x28 pixels), so
### should use either 88,290 (327x270) or 19,600 (140x140), possibly x3 for the
### RGB channels

import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Encoder(nn.Module):
    def __init__(self, inputSize, latent):
        super(Encoder, self).__init__()
        self.linear = nn.Linear(inputSize, latent)
        self.linear2 = nn.Linear(latent, int(latent/4))
        self.linear3 = nn.Linear(int(latent/4), int(latent/16))
        self.linear4 = nn.Linear(int(latent/4), int(latent/16))

        self.Norm = torch.distributions.Normal(0, 1)
        self.Norm.loc = self.Norm.loc.cuda()
        self.Norm.scale = self.Norm.scale.cuda()
        self.KL = 0

    def forward(self, data):
        data = torch.flatten(data, start_dim=1)
        data = torch.nn.functional.relu(self.linear(data))
        data = torch.nn.functional.relu(self.linear2(data))
        mu =  self.linear3(data)
        sigma = torch.exp(self.linear4(data))
        z = mu + sigma * self.Norm.sample(mu.shape)
        self.KL = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()

        return z


class Decoder(nn.Module):
    def __init__(self, inputSize, latent):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(540, latent)
        self.linear2 = nn.Linear(latent, inputSize)

    def forward(self, latentRep):
        latentRep = torch.nn.functional.relu(self.linear(latentRep))
        output = torch.sigmoid(self.linear2(latentRep))

        return output.reshape((-1, 3, 48, 48))


class VAE(nn.Module):
    def __init__(self, inputSize, latent, encoders=5):
        super(VAE, self).__init__()
        self.encoders = nn.ModuleList([Encoder(inputSize, latent) for x in range(encoders)])
        self.decoder = Decoder(inputSize, latent)

    def forward(self, data):
        latentReps = []#nn.ModuleList()

        ### Each encoder encodes its respective ith consecutive inputs
        for encoder in range(len(self.encoders)):
            #print(data[encoder])
            latentReps.append(self.encoders[encoder](data[encoder]))

        conc = torch.cat(latentReps, 1)
        output = self.decoder(conc)

        return output


In [ ]:
#@title 2 - VAE Training Loop
### Training loop for VAE
def trainVAE(vae, data, epochs=20):
    ### Automatic Mixed Precision - reduce to float16 to save space.
    scaler = torch.cuda.amp.GradScaler()

    optimiser = torch.optim.Adam(vae.parameters())

    for epoch in range(epochs):
        for i, data in enumerate(trainDataloader):

            ### Batches of x targets and 5 lists of x inputs - corresponding rows
            targets, inputs = data
            targets = targets.to(device)
            inputs = [inputs[x].to(device) for x in range(len(inputs))]

            optimiser.zero_grad()

            ### Try amp with bfloat16 too
            with torch.amp.autocast(device_type=device, dtype=torch.float16):
                predicted = vae(inputs)
                sumKLs = 0
                for encoder in range(len(vae.encoders)):
                    sumKLs += vae.encoders[encoder].KL
                sumKLs = sumKLs / len(vae.encoders)
                loss = ((targets - predicted)**2).sum() + sumKLs

            scaler.scale(loss).backward()
            scaler.step(optimiser)
            scaler.update()

        predicted = predicted.byte()
        plt.imshow(predicted[0].permute(1, 2, 0))
        plt.show()
        print(loss)

dims = 6912
hidden = int(dims / 4)
vae = VAE(dims, hidden).to(device)
trainVAE(vae, trainDataloader, 5)

NameError: name 'VAE' is not defined